<a href="https://colab.research.google.com/github/LoggingNewMemory/MirrorBot/blob/main/Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Google Colab Mirrorbot** <br />
By: Kanagawa Yamada

In [26]:
# 🛠️ Install required packages
!pip install requests beautifulsoup4 pyrogram tgcrypto


In [27]:
# 📦 Import necessary libraries
import os
import requests
import asyncio
from urllib.parse import urlparse
from pathlib import Path
import zipfile
import tarfile
from google.colab import drive, files
from IPython.display import display, HTML
import shutil


In [28]:
# 🤖 MirrorBot Class: Initialization and Setup
class MirrorBot:
    def __init__(self):
        self.download_dir = "/content/downloads"
        self.drive_dir = "/content/drive/MyDrive/MirrorBot"
        self.mount_drive()
        self.setup_directories()

    def mount_drive(self):
        """Mount Google Drive"""
        try:
            drive.mount('/content/drive')
            print("✅ Google Drive mounted successfully")
        except Exception as e:
            print(f"❌ Error mounting drive: {e}")

    def setup_directories(self):
        """Create necessary directories"""
        os.makedirs(self.download_dir, exist_ok=True)
        os.makedirs(self.drive_dir, exist_ok=True)
        print(f"📁 Directories created: {self.download_dir}, {self.drive_dir}")

    # 🔧 MirrorBot Class: Helper Methods

    def get_filename_from_url(self, url):
        parsed = urlparse(url)
        filename = os.path.basename(parsed.path)
        if not filename or '.' not in filename:
            filename = f"download_{hash(url) % 10000}"
        return filename

    def format_size(self, bytes):
        for unit in ['B', 'KB', 'MB', 'GB']:
            if bytes < 1024:
                return f"{bytes:.1f} {unit}"
            bytes /= 1024
        return f"{bytes:.1f} TB"

    # 📥 Download and Extract
    def download_file(self, url, custom_name=None):
        try:
            print(f"🔄 Starting download: {url}")
            headers = {'User-Agent': 'Mozilla/5.0'}
            response = requests.get(url, headers=headers, stream=True)
            response.raise_for_status()

            filename = custom_name or self.get_filename_from_url(url)
            filepath = os.path.join(self.download_dir, filename)
            total_size = int(response.headers.get('content-length', 0))
            downloaded = 0

            with open(filepath, 'wb') as file:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        file.write(chunk)
                        downloaded += len(chunk)
                        if total_size > 0:
                            progress = (downloaded / total_size) * 100
                            print(f"\r📥 Progress: {progress:.1f}%", end='', flush=True)

            print(f"\n✅ Downloaded: {filename} ({self.format_size(downloaded)})")
            return filepath

        except Exception as e:
            print(f"❌ Download failed: {e}")
            return None

    def extract_archive(self, filepath):
        try:
            extract_dir = os.path.splitext(filepath)[0]
            os.makedirs(extract_dir, exist_ok=True)

            if filepath.endswith(('.zip', '.jar')):
                with zipfile.ZipFile(filepath, 'r') as zip_ref:
                    zip_ref.extractall(extract_dir)
            elif filepath.endswith(('.tar', '.tar.gz', '.tgz')):
                with tarfile.open(filepath, 'r:*') as tar_ref:
                    tar_ref.extractall(extract_dir)
            else:
                print("⚠️ Not a supported archive format")
                return filepath

            print(f"📦 Extracted to: {extract_dir}")
            return extract_dir

        except Exception as e:
            print(f"❌ Extraction failed: {e}")
            return filepath

        # 📁 File Operations
    def move_to_drive(self, filepath):
        try:
            filename = os.path.basename(filepath)
            drive_path = os.path.join(self.drive_dir, filename)

            if os.path.isdir(filepath):
                shutil.copytree(filepath, drive_path, dirs_exist_ok=True)
            else:
                shutil.copy2(filepath, drive_path)

            print(f"☁️ Moved to Drive: {drive_path}")
            return drive_path

        except Exception as e:
            print(f"❌ Failed to move to Drive: {e}")
            return None

    def list_downloads(self):
        print("\n📋 Downloaded Files:")
        print("-" * 50)
        for root, dirs, files in os.walk(self.download_dir):
            level = root.replace(self.download_dir, '').count(os.sep)
            indent = ' ' * 2 * level
            print(f"{indent}{os.path.basename(root)}/")
            subindent = ' ' * 2 * (level + 1)
            for file in files:
                filepath = os.path.join(root, file)
                size = os.path.getsize(filepath)
                print(f"{subindent}{file} ({self.format_size(size)})")

    def clear_downloads(self):
        try:
            shutil.rmtree(self.download_dir)
            os.makedirs(self.download_dir, exist_ok=True)
            print("🗑️ Downloads cleared")
        except Exception as e:
            print(f"❌ Error clearing downloads: {e}")

    # 🚀 Main Mirror Function
    def mirror(self, url, extract=False, move_to_drive=True, custom_name=None):
        print(f"🚀 Mirror Bot Starting...")
        print(f"📂 Download directory: {self.download_dir}")
        print(f"☁️ Drive directory: {self.drive_dir}")
        print("-" * 50)

        filepath = self.download_file(url, custom_name)
        if not filepath:
            return None

        if extract:
            filepath = self.extract_archive(filepath)

        if move_to_drive:
            drive_path = self.move_to_drive(filepath)
            if drive_path:
                print(f"✨ Mirror completed! File saved to Drive")
                return drive_path

        print(f"✨ Mirror completed! File saved locally")
        return filepath

      # ✅ Initialize and Provide Usage Functions
    bot = MirrorBot()

    def mirror_file(url, extract=False, move_to_drive=True, custom_name=None):
      return bot.mirror(url, extract, move_to_drive, custom_name)

    def list_files():
      bot.list_downloads()

    def clear_files():
      bot.clear_downloads()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive mounted successfully
📁 Directories created: /content/downloads, /content/drive/MyDrive/MirrorBot


In [ ]:
# 📘 Display Usage Instructions
display(HTML("""
<div style="border: 2px solid #4CAF50; padding: 15px; border-radius: 10px; background-color: #f9f9f9;">
<h3>🤖 Mirror Bot Ready!</h3>
<h4>Usage Examples:</h4>
<code>
# Mirror a file<br>
mirror_file("https://example.com/file.zip")<br><br>

# Mirror and extract archive<br>
mirror_file("https://example.com/archive.zip", extract=True)<br><br>

# Mirror with custom name<br>
mirror_file("https://example.com/file.pdf", custom_name="my_document.pdf")<br><br>

# Mirror without moving to Drive<br>
mirror_file("https://example.com/file.txt", move_to_drive=False)<br><br>

# List downloaded files<br>
list_files()<br><br>

# Clear downloads<br>
clear_files()
</code>
</div>
"""))


Test